In [34]:
from IPython.display import display, HTML
from google.colab import output
import time

# 사용자로부터 SERVER_URL 입력 받기
SERVER_URL = input("Please enter the SERVER_URL: ")

game_code = """
<div style="display: flex; align-items: flex-start;">
  <div>
    <canvas id="gameCanvas" width="400" height="400" style="border:1px solid #000000;"></canvas>
    <div id="inputContainer" style="display:none;">
      <input type="text" id="chatInput" style="width: 300px;">
    </div>
    <div id="healthBar" style="width: 200px; height: 20px; background-color: #ddd; margin-top: 10px;">
      <div id="health" style="width: 100%; height: 100%; background-color: green;"></div>
    </div>
    <div id="enemyHealthBar" style="width: 200px; height: 20px; background-color: #ddd; margin-top: 10px;">
      <div id="enemyHealth" style="width: 100%; height: 100%; background-color: red;"></div>
    </div>
    <div id="levelInfo" style="margin-top: 10px;">Level: 1</div>
  </div>
  <div id="goapInfo" style="margin-left: 20px; width: 200px; font-family: Arial, sans-serif; font-size: 12px;"></div>
</div>

<script>
const canvas = document.getElementById('gameCanvas');
const ctx = canvas.getContext('2d');
const chatInput = document.getElementById('chatInput');
const inputContainer = document.getElementById('inputContainer');
const healthBar = document.getElementById('health');
const enemyHealthBar = document.getElementById('enemyHealth');
const levelInfo = document.getElementById('levelInfo');


//서버 통신
let SERVER_URL = '';

async function sendToServer(data) {
  try {
    const response = await fetch(`${SERVER_URL}/api/game`, {
      method: 'POST',
      headers: {
        'Content-Type': 'application/json',
      },
      body: JSON.stringify(data),
    });
    return await response.json();
  } catch (error) {
    console.error('Error sending data to server:', error);
  }
}

async function receiveFromServer() {
  try {
    const response = await fetch(`${SERVER_URL}/api/game`);
    return await response.json();
  } catch (error) {
    console.error('Error receiving data from server:', error);
  }
}

async function sendUserInputToServer(input) {
  const gameState = {
    npcGoal: goap.currentGoal,
    playerHealth: player.health,
    npcHealth: npc.health,
    enemyHealth: enemy.health,
    playerPosition: { x: player.x, y: player.y },
    npcPosition: { x: npc.x, y: npc.y },
    enemyPosition: { x: enemy.x, y: enemy.y },
    userInput: input
  };

  const serverResponse = await sendToServer(gameState);
  handleServerResponse(serverResponse);

  // After handling the server response, exit input mode
  isInputMode = false;
  inputContainer.style.display = 'none';
}

function handleServerResponse(response) {
  if (response.npcDialogue) {
   showMessage(response.npcDialogue, npc);
  }

  if (response.npcRoles) {
    updateNPCRoles(response.npcRoles);
  }
}

function updateNPCRoles(roles) {
  initializeGOAP(roles);
}






//객체 정의
const boxSize = 20;
let dx = 0;
let dy = 0;
const speed = 2;

let isInputMode = false;
let currentMessage = '';
let messageTimer = 0;

let health = 100;
let isGameOver = false;

const fence = {
  x: 50,
  y: 50,
  width: 150,
  height: 100,
  gateWidth: 30,
  gateHeight: 5
};

let player = {
  x: 100,
  y: 100,
  size: 20,
  speed: 0.9,
  health: 100,
  maxHealth: 100,
  lastHitTime: 0,
  healRate: 0.05,
  moveCost: 0.02,
  attackCost: 0.5,
  moveCost: 0.1,
  currentMessage: '',
  messageTimer: 0
};

let npc = {
  x: 0,
  y: 0,
  size: 20,
  speed: 0.7,
  health: 100,
  maxHealth: 100,
  color: 'purple',
  currentMessage: '',
  messageTimer: 0,
  lastHitTime: 0,
  healRate: 0.03,
  moveCost: 0.02,  // 이 값을 조정하여 NPC의 이동에 따른 체력 감소량을 조절할 수 있습니다.
  attackCost: 0.3,
  attackDamage: 1 // NPC의 공격력
};

let enemy = {
  x: 400,
  y: 400,
  size: 15,
  speed: 0.5,
  dx: 0,
  dy: 0,
  detectionRadius: 150,
  changeDirectionInterval: 3000,
  lastDirectionChange: 0,
  health: 50,
  maxHealth: 50,
  damage: 10,
  level: 1,
  attackInterval: 1000,
  lastAttackTime: 0,
  currentMessage: '',
  messageTimer: 0
};



// RRT 노드 클래스
class RRTNode {
  constructor(x, y) {
    this.x = x;
    this.y = y;
    this.parent = null;
  }
}

// 간단한 RRT 구현
function simpleRRT(start, goal, obstacles, maxIterations = 1000) {
  let nodes = [new RRTNode(start.x, start.y)];

  for (let i = 0; i < maxIterations; i++) {
    let randomPoint = { x: Math.random() * canvas.width, y: Math.random() * canvas.height };
    let nearestNode = findNearestNode(randomPoint, nodes);
    let newNode = extendTowards(nearestNode, randomPoint);

    if (!collidesWith(newNode, obstacles)) {
      nodes.push(newNode);
      if (distanceBetween(newNode, goal) < 10) {
        return reconstructPath(newNode);
      }
    }
  }

  return null; // 경로를 찾지 못함
}

// 헬퍼 함수들
function findNearestNode(point, nodes) {
  return nodes.reduce((nearest, node) =>
    distanceBetween(point, node) < distanceBetween(point, nearest) ? node : nearest
  );
}

function extendTowards(fromNode, toPoint) {
  const MAX_EXTEND_LENGTH = 10;
  const dx = toPoint.x - fromNode.x;
  const dy = toPoint.y - fromNode.y;
  const distance = Math.sqrt(dx*dx + dy*dy);

  const newNode = new RRTNode(
    fromNode.x + (dx / distance) * Math.min(MAX_EXTEND_LENGTH, distance),
    fromNode.y + (dy / distance) * Math.min(MAX_EXTEND_LENGTH, distance)
  );
  newNode.parent = fromNode;

  return newNode;
}

function collidesWith(node, obstacles) {
  return obstacles.some(obstacle =>
    node.x >= obstacle.x && node.x <= obstacle.x + obstacle.width &&
    node.y >= obstacle.y && node.y <= obstacle.y + obstacle.height
  );
}

function distanceBetween(point1, point2) {
  const dx = point1.x - point2.x;
  const dy = point1.y - point2.y;
  return Math.sqrt(dx*dx + dy*dy);
}

function reconstructPath(endNode) {
  let path = [];
  let currentNode = endNode;
  while (currentNode !== null) {
    path.unshift({x: currentNode.x, y: currentNode.y});
    currentNode = currentNode.parent;
  }
  return path;
}

function calculatePathDistance(path) {
  let distance = 0;
  for (let i = 1; i < path.length; i++) {
    distance += distanceBetween(path[i-1], path[i]);
  }
  return distance;
}

// GOAP 정보 업데이트 함수
function updateGOAPInfo() {
  const goapInfoElement = document.getElementById('goapInfo');
  let infoHTML = `<h3>GOAP Info</h3>`;
  infoHTML += `<p><strong>Current Goal:</strong> ${goap.currentGoal}</p>`;

  if (goap.goalCosts) {
    for (const [goal, costs] of Object.entries(goap.goalCosts)) {
      infoHTML += `<h4>${goal}</h4>`;
      infoHTML += `<p>Movement: ${costs.movement.toFixed(2)}</p>`;
      infoHTML += `<p>Attack: ${costs.attack.toFixed(2)}</p>`;
      infoHTML += `<p>Total: ${costs.total.toFixed(2)}</p>`;
    }
  }

  goapInfoElement.innerHTML = infoHTML;
}

// 목표 선택 및 경로 계획
function selectGoalAndPlan() {
  const goals = ['combat', 'rest', 'explore', 'communicate'];
  let bestGoal = null;
  let bestScore = -Infinity;
  let goalCosts = {};

  for (const goal of goals) {
    const targetPosition = getGoalPosition(goal);
    const path = simpleRRT({x: npc.x, y: npc.y}, targetPosition, [enemy, fence]);

    if (path) {
      const distance = calculatePathDistance(path);
      const movementCost = distance * npc.moveCost;

      let attackCost = 0;
      if (goal === 'combat') {
        const estimatedAttacks = Math.ceil(enemy.health / 10); // Assuming 10 damage per attack
        attackCost = estimatedAttacks * npc.attackCost;
      }

      const totalCost = movementCost + attackCost;
      const goalScore = calculateGoalScore(goal, totalCost);

      goalCosts[goal] = {
        movement: movementCost,
        attack: attackCost,
        total: totalCost
      };

      if (goalScore > bestScore) {
        bestScore = goalScore;
        bestGoal = goal;
      }
    } else {
      goalCosts[goal] = {
        movement: Infinity,
        attack: 0,
        total: Infinity
      };
    }
  }

  if (bestGoal) {
    goap.currentGoal = bestGoal;
    goap.targetPosition = getGoalPosition(bestGoal);
  }

  goap.goalCosts = goalCosts; // Store the goal costs in the goap object
  updateGOAPInfo(); // Update the GOAP info display
}

function calculateGoalScore(goal, healthCost) {
  const baseScore = goap.roles[goal];
  const healthPercentage = npc.health / npc.maxHealth;

  // 체력이 낮을 때 'rest' 목표의 점수를 높임
  if (goal === 'rest' && healthPercentage < 0.5) {
    return baseScore * (2 - healthPercentage) - healthCost;
  }

  // 체력이 높을 때 'combat' 목표의 점수를 높임
  if (goal === 'combat' && healthPercentage > 0.7) {
    return baseScore * healthPercentage - healthCost;
  }

  return baseScore - healthCost;
}

// 주기적으로 목표 선택 및 경로 계획 실행
setInterval(selectGoalAndPlan, 3000); // 3초마다 실행






// 기존의 goap 객체 정의를 다음과 같이 수정
let goap = {
  roles: {combat: 0, rest: 0, explore: 0, communicate: 0},
  currentGoal: null,
  plan: [],
  planIndex: 0,
  goalCosts: {} // Add this line to store goal costs
};
function initializeGOAP(roles) {
  goap.roles = roles;
  goap.currentGoal = selectGoal();
  goap.plan = generateInitialPlan();
  goap.planIndex = 0;
}

function selectGoal() {
  const goals = Object.entries(goap.roles);
  goals.sort((a, b) => b[1] - a[1]);
  return goals[0][0];
}

function generateInitialPlan() {
  return Array(10).fill().map(() => ({
    x: Math.random() * canvas.width,
    y: Math.random() * canvas.height
  }));
}


// 업데이트
function updatePositions() {
  if (isGameOver || isInputMode) return;

  // Update player
  updatePlayer();

  // Update enemy position and attack
  updateEnemy();
  enemyAttack();

  // Update NPC
  updateNPC();

  // Heal player and NPC if not recently hit
  healEntity(player);
  healEntity(npc);

  // Update health bars
  updateHealthBars();

  if (messageTimer > 0) {
    messageTimer--;
  }
}

function updatePlayer() {
  const newPlayerX = player.x + dx;
  const newPlayerY = player.y + dy;

  if (canPlayerMove(newPlayerX, newPlayerY)) {
    const distanceMoved = Math.sqrt((newPlayerX - player.x)**2 + (newPlayerY - player.y)**2);
    player.health = Math.max(0, player.health - player.moveCost * distanceMoved);
    player.x = newPlayerX;
    player.y = newPlayerY;
  }

  player.x = Math.max(0, Math.min(player.x, canvas.width - player.size));
  player.y = Math.max(0, Math.min(player.y, canvas.height - player.size));

}

function updateEnemy() {
  const playerDist = getDistance(enemy, player);
  const npcDist = getDistance(enemy, npc);
  let target = playerDist <= npcDist ? player : npc;

  if (getDistance(enemy, target) <= enemy.detectionRadius) {
    const dx = target.x - enemy.x;
    const dy = target.y - enemy.y;
    const distance = Math.sqrt(dx * dx + dy * dy);
    enemy.dx = (dx / distance) * enemy.speed;
    enemy.dy = (dy / distance) * enemy.speed;
  } else {
    const now = Date.now();
    if (now - enemy.lastDirectionChange > enemy.changeDirectionInterval) {
      const angle = Math.random() * 2 * Math.PI;
      enemy.dx = Math.cos(angle) * enemy.speed;
      enemy.dy = Math.sin(angle) * enemy.speed;
      enemy.lastDirectionChange = now;
    }
  }

  const newX = enemy.x + enemy.dx;
  const newY = enemy.y + enemy.dy;

  if (!isInsideFence(enemy.x, enemy.y) && !isColliding(newX, newY, enemy.size, enemy.size,
      fence.x, fence.y, fence.width, fence.height)) {
    enemy.x = newX;
    enemy.y = newY;
  } else {
    enemy.dx = -enemy.dx;
    enemy.dy = -enemy.dy;
  }

  enemy.x = Math.max(0, Math.min(enemy.x, canvas.width - enemy.size));
  enemy.y = Math.max(0, Math.min(enemy.y, canvas.height - enemy.size));
}


function enemyAttack() {
  const now = Date.now();
  if (now - enemy.lastAttackTime < enemy.attackInterval) return;

  const playerDist = getDistance(enemy, player);
  const npcDist = getDistance(enemy, npc);
  let target = playerDist <= npcDist ? player : npc;

  if (getDistance(enemy, target) <= enemy.size + target.size) {
    target.health = Math.max(0, target.health - enemy.damage);
    target.lastHitTime = now;
    enemy.lastAttackTime = now;

    if (target.health <= 0) {
      isGameOver = true;
    }
  }
}

function updateNPC() {
  if (goap.plan.length > 0 && !isInputMode) {

    let target = getGoalPosition();
    let dx = target.x - npc.x;
    let dy = target.y - npc.y;
    let distance = Math.sqrt(dx*dx + dy*dy);

    if (distance < npc.speed) {
      showMessage("npc 좋겠네", enemy);
      selectGoalAndPlan()
    } else {
      let moveX = (dx / distance) * npc.speed;
      let moveY = (dy / distance) * npc.speed;
      let distanceMoved = Math.sqrt(moveX*moveX + moveY*moveY);

      npc.x += moveX;
      npc.y += moveY;
      npc.health = Math.max(0, npc.health - npc.moveCost * distanceMoved);
    }


    switch (goap.currentGoal) {
      case 'combat':
        if (isColliding(npc.x, npc.y, npc.size, npc.size, enemy.x, enemy.y, enemy.size, enemy.size)) {
          enemy.health -= 1;
          npc.health = Math.max(0, npc.health - npc.attackCost);
          npc.lastHitTime = Date.now();  // 공격 시 마지막 피격 시간 업데이트
          enemyHealthBar.style.width = (enemy.health / enemy.maxHealth * 100) + "%";
          if (enemy.health <= 0) {
            levelUp();
          }
        }
        break;
      case 'rest':
        if (isInsideFence(npc.x, npc.y)) {
          npc.health = Math.min(npc.health + npc.healRate * 2, npc.maxHealth);  // 휴식 시 더 빠른 회복
        }
        break;
      case 'communicate':
        if (isNearPlayer(npc.x, npc.y)) {
          showMessage("너네 이야기하지 마라", enemy);
        }
        break;
    }
  }

  // NPC의 체력이 0 이하가 되면 게임 오버
  if (npc.health <= 0) {
    isGameOver = true;
    showMessage("NPC가 사망했습니다. 게임 오버!", npc);
  }
}



function healEntity(entity) {
  const now = Date.now();
  if (now - entity.lastHitTime > 5000) {  // 5초 동안 공격받지 않았다면
    entity.health = Math.min(entity.maxHealth, entity.health + entity.healRate);
  }
}

function updateHealthBars() {
  healthBar.style.width = (player.health / player.maxHealth * 100) + "%";
  enemyHealthBar.style.width = (enemy.health / enemy.maxHealth * 100) + "%";
  // NPC 체력 바 업데이트 (캔버스에 직접 그리는 방식)
}
function getDistance(entity1, entity2) {
  const dx = entity1.x - entity2.x;
  const dy = entity1.y - entity2.y;
  return Math.sqrt(dx * dx + dy * dy);
}

function attack() {
  if (isColliding(player.x, player.y, player.size, player.size, enemy.x, enemy.y, enemy.size, enemy.size)) {
    enemy.health -= 10;
    player.health = Math.max(0, player.health - player.attackCost);
    enemyHealthBar.style.width = (enemy.health / enemy.maxHealth * 100) + "%";
    if (enemy.health <= 0) {
      levelUp();
    }
  }
}



//화면 그리기
function drawGame() {
  ctx.clearRect(0, 0, canvas.width, canvas.height);

  // Draw fence
  ctx.fillStyle = "brown";
  ctx.fillRect(fence.x, fence.y, fence.width, fence.gateHeight);
  ctx.fillRect(fence.x, fence.y, fence.gateHeight, fence.height);
  ctx.fillRect(fence.x, fence.y + fence.height - fence.gateHeight, fence.width, fence.gateHeight);
  ctx.fillRect(fence.x + fence.width - fence.gateHeight, fence.y, fence.gateHeight, fence.height);

  // Draw gate
  ctx.fillStyle = "green";
  ctx.fillRect(fence.x + fence.width/2 - fence.gateWidth/2, fence.y + fence.height - fence.gateHeight, fence.gateWidth, fence.gateHeight);

  // Draw enemy
  ctx.fillStyle = "red";
  ctx.fillRect(enemy.x, enemy.y, enemy.size, enemy.size);

  // Draw player
  ctx.fillStyle = "blue";
  ctx.fillRect(player.x, player.y, player.size, player.size);

  // Draw NPC
  ctx.fillStyle = npc.color;
  ctx.fillRect(npc.x, npc.y, npc.size, npc.size);

  // Draw NPC health bar
  ctx.fillStyle = "lightgray";
  ctx.fillRect(npc.x, npc.y - 10, npc.size, 5);
  ctx.fillStyle = "green";
  ctx.fillRect(npc.x, npc.y - 10, npc.size * (npc.health / npc.maxHealth), 5);

  // Draw speech bubbles
  drawSpeechBubble(player.x, player.y, player.size, player.currentMessage, player.messageTimer);
  drawSpeechBubble(npc.x, npc.y, npc.size, npc.currentMessage, npc.messageTimer);
  drawSpeechBubble(enemy.x, enemy.y, enemy.size, enemy.currentMessage, enemy.messageTimer);

  // Draw game over message
  if (isGameOver) {
    ctx.fillStyle = "rgba(0, 0, 0, 0.5)";
    ctx.fillRect(0, 0, canvas.width, canvas.height);
    ctx.fillStyle = "white";
    ctx.font = "30px Arial";
    ctx.textAlign = "center";
    ctx.fillText("Game Over", canvas.width / 2, canvas.height / 2);
  }
}

function drawSpeechBubble(entityX, entityY, entitySize, message, timer) {
  if (message && timer > 0) {
    const padding = 5;
    const bubbleHeight = 30;
    ctx.font = '12px Arial';
    const textWidth = ctx.measureText(message).width;
    const bubbleWidth = textWidth + padding * 2;

    ctx.fillStyle = 'white';
    ctx.strokeStyle = 'black';
    ctx.beginPath();
    ctx.moveTo(entityX + entitySize / 2, entityY - 5);
    ctx.lineTo(entityX + entitySize / 2 - 5, entityY - 15);
    ctx.lineTo(entityX + entitySize / 2 + 5, entityY - 15);
    ctx.closePath();
    ctx.fill();
    ctx.stroke();

    ctx.beginPath();
    ctx.roundRect(entityX + entitySize / 2 - bubbleWidth / 2, entityY - bubbleHeight - 15, bubbleWidth, bubbleHeight, 5);
    ctx.fill();
    ctx.stroke();

    ctx.fillStyle = 'black';
    ctx.fillText(message, entityX + entitySize / 2 - textWidth / 2, entityY - bubbleHeight + 5);
  }
}

function isInsideFence(checkX, checkY) {
  return checkX >= fence.x && checkX <= fence.x + fence.width &&
         checkY >= fence.y && checkY <= fence.y + fence.height;
}



// 조작하기
function canPlayerMove(newX, newY) {
  const gateLeft = fence.x + fence.width/2 - fence.gateWidth/2;
  const gateRight = gateLeft + fence.gateWidth;
  const gateTop = fence.y + fence.height - fence.gateHeight;

  if (isInsideFence(newX, newY) ||
      (newY + player.size > gateTop && newX + player.size > gateLeft && newX < gateRight)) {
    return true;
  }

  if (!isInsideFence(player.x, player.y) && !isColliding(newX, newY, player.size, player.size,
      fence.x, fence.y, fence.width, fence.height)) {
    return true;
  }

  return false;
}

function isColliding(x1, y1, w1, h1, x2, y2, w2, h2) {
  return x1 < x2 + w2 && x1 + w1 > x2 && y1 < y2 + h2 && y1 + h1 > y2;
}


function handleKeyDown(e) {
  if (isGameOver) return;

  if (isInputMode) {
    if (e.key === 'Enter') {
      showMessage(chatInput.value, player);
      sendUserInputToServer(chatInput.value);
      // Exit input mode and hide the input container
      isInputMode = false;
      inputContainer.style.display = 'none';
      chatInput.value = ''; // Clear the input
    }
    return;
  }

  switch(e.key.toLowerCase()) {
    case "w": dy = -speed; break;
    case "s": dy = speed; break;
    case "a": dx = -speed; break;
    case "d": dx = speed; break;
    case " ": attack(); break;
    case "enter":
      isInputMode = true;
      inputContainer.style.display = 'block';
      chatInput.focus();
      break;
  }
}

function handleKeyUp(e) {
  if (isInputMode) return;

  switch(e.key.toLowerCase()) {
    case "w":
    case "s": dy = 0; break;
    case "a":
    case "d": dx = 0; break;
  }
}



// 이외 잡다한 남은거
function levelUp() {
  enemy.level++;
  enemy.maxHealth += 20;
  enemy.health = enemy.maxHealth;
  enemy.damage += 0.2;
  enemy.speed += 0.1;
  enemyHealthBar.style.width = "100%";
  levelInfo.textContent = `Level: ${enemy.level}`;

  spawnNewEnemy();
}

function spawnNewEnemy() {
  const side = Math.floor(Math.random() * 4);
  switch(side) {
    case 0:
      enemy.x = Math.random() * (canvas.width - enemy.size);
      enemy.y = 0;
      break;
    case 1:
      enemy.x = canvas.width - enemy.size;
      enemy.y = Math.random() * (canvas.height - enemy.size);
      break;
    case 2:
      enemy.x = Math.random() * (canvas.width - enemy.size);
      enemy.y = canvas.height - enemy.size;
      break;
    case 3:
      enemy.x = 0;
      enemy.y = Math.random() * (canvas.height - enemy.size);
      break;
  }
}




function showMessage(message, entity) {
  entity.currentMessage = message;
  entity.messageTimer = 200;
}

function isNearPlayer(x, y) {
  const distance = Math.sqrt((x - player.x)**2 + (y - player.y)**2);
  return distance < 50;
}

function getGoalPosition() {
  switch (goap.currentGoal) {
    case 'combat': return {x: enemy.x, y: enemy.y};
    case 'rest': return {
      x: fence.x + fence.width / 2,
      y: fence.y + fence.height / 2
    };
    case 'explore': return {
      x: Math.random() * canvas.width,
      y: Math.random() * canvas.height
    };
    case 'communicate': return {x: player.x, y: player.y};
  }
}

document.addEventListener('keydown', handleKeyDown);
document.addEventListener('keyup', handleKeyUp);



//메인 루프
function gameLoop() {
  updatePositions();
  drawGame();
  updateGOAPInfo(); // GOAP 정보 업데이트
  requestAnimationFrame(gameLoop);
}

function initGame() {
  initializeGOAP({combat: 3, rest: 6, explore: 3, communicate: 3});
  gameLoop();
}

initGame();
"""

# HTML 표시
display(HTML(game_code))

# 게임 초기화 및 SERVER_URL 설정
output.eval_js(f'SERVER_URL = "{SERVER_URL}"; initGame();')

# 게임 실행 (60초 동안)
start_time = time.time()
while time.time() - start_time < 60:
    time.sleep(0.1)
    output.eval_js('updatePositions(); drawGame();')

print("게임 종료")

KeyboardInterrupt: Interrupted by user